In [19]:
import plotly.express as px
import pandas as pd
import json
px.set_mapbox_access_token("pk.eyJ1IjoicGFycXVhciIsImEiOiJja3lpcXMycGUxbmF5MnBzZXVzMHBzaXl4In0.jz0tx-HTJWym8jWPa8lqiA")
import googlemaps
from concurrent.futures import ThreadPoolExecutor
from glob import glob 
pd.options.display.max_columns = None

from post_roe.query import BucketQuery as bq


with open("api.txt", "r") as f:
    places_api_key = f.read()

zip3 = bq.census_zip3_query()
gmaps = googlemaps.Client(key=places_api_key)
# fetch 

def fetch_places(loc: tuple, phrase="abortion clinic", radius=1000):
    response = gmaps.places(phrase,location=loc,radius=1000)
    return response

def _cache_places(i) -> None:
    # i = 0
    response = fetch_places(zip3['lat_lon'][i])
    origin_label = zip3['zip3'][i]
    with open(f"clinics/{origin_label}.json", "w") as f:
        json.dump(response, f)

def cache_all():
    # mess w/ the range
    with ThreadPoolExecutor(max_workers=20) as exec:
        futures = exec.map(_cache_places, range(500, 966))

In [53]:
paths = glob("/Users/parker/Development/post-roe/data/clinics/*")
def _open(path):
    with open(path) as f: 
        data = json.load(f)
    data['zip3'] = path.split("/")[-1].split('.')[0]
    return data

places = pd.DataFrame([_open(p) for p in paths])

clinics = pd.json_normalize(places['results'].explode())
clinics = clinics.drop_duplicates(subset=['place_id'])
clinics['lat'] = clinics['geometry.location.lat']
clinics['lon'] = clinics['geometry.location.lng']
clinics

,business_status,formatted_address,icon,icon_background_color,icon_mask_base_uri,name,photos,place_id,rating,reference,types,user_ratings_total,geometry.location.lat,geometry.location.lng,geometry.viewport.northeast.lat,geometry.viewport.northeast.lng,geometry.viewport.southwest.lat,geometry.viewport.southwest.lng,opening_hours.open_now,plus_code.compound_code,plus_code.global_code,lat,lon
0,OPERATIONAL,"69 Newman Springs Rd, Shrewsbury, NJ 07702, Un...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,Planned Parenthood - Shrewsbury Health Center,"[{'height': 681, 'html_attributions': ['<a hre...",ChIJW7V0cy8wwokRpIOT8AQedpI,2.6,ChIJW7V0cy8wwokRpIOT8AQedpI,"[hospital, health, point_of_interest, establis...",48.0,40.337842,-74.066505,40.339145,-74.065241,40.336446,-74.067941,False,"8WQM+49 Shrewsbury, New Jersey",87G78WQM+49,40.337842,-74.066505
1,OPERATIONAL,"14 DeKalb Ave 4th Floor, Brooklyn, NY 11201, U...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,Brooklyn Abortion Clinic,"[{'height': 3456, 'html_attributions': ['<a hr...",ChIJo8-hRbNbwokRqa9WiN6gi2c,4.1,ChIJo8-hRbNbwokRqa9WiN6gi2c,"[doctor, health, point_of_interest, establishm...",149.0,40.689721,-73.982176,40.691197,-73.980788,40.688498,-73.983487,False,M2Q9+V4 New York,87G8M2Q9+V4,40.689721,-73.982176
2,OPERATIONAL,"102 Candlewood Commons, Howell Township, NJ 07...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,Options,"[{'height': 3024, 'html_attributions': ['<a hr...",ChIJvVfM8Ht_wYkR-id8DfnWmas,3.7,ChIJvVfM8Ht_wYkR-id8DfnWmas,"[health, point_of_interest, establishment]",6.0,40.147447,-74.224413,40.148825,-74.223069,40.146125,-74.225769,NaN,"4QWG+X6 Howell Township, New Jersey",87G74QWG+X6,40.147447,-74.224413
3,OPERATIONAL,"228 Main St, Woodbridge Township, NJ 07095, Un...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,American Womens Services - Woodbridge,"[{'height': 1288, 'html_attributions': ['<a hr...",ChIJG345wbe1w4kRsFve_tDZhWg,2.6,ChIJG345wbe1w4kRsFve_tDZhWg,"[health, point_of_interest, establishment]",40.0,40.552115,-74.287140,40.553445,-74.285832,40.550746,-74.288532,False,"HP27+R4 Woodbridge Township, New Jersey",87G7HP27+R4,40.552115,-74.287140
4,OPERATIONAL,"6721 Avenue U, Brooklyn, NY 11234, United States",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,Brooklyn Women’s Gynecological & Abortion Serv...,"[{'height': 1702, 'html_attributions': ['<a hr...",ChIJ3X4f5TZDwokRaLrUDEWUB80,3.4,ChIJ3X4f5TZDwokRaLrUDEWUB80,"[health, point_of_interest, establishment]",10.0,40.618421,-73.909974,40.619793,-73.908589,40.617094,-73.911288,False,"J39Q+9X Brooklyn, New York",87G8J39Q+9X,40.618421,-73.909974
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11800,OPERATIONAL,"101 Dudley St, Providence, RI 02905, United St...",https://maps.gstatic.com/mapfiles/place_api/ic...,#F88181,https://maps.gstatic.com/mapfiles/place_api/ic...,Women & Infants Hospital,"[{'height': 6936, 'html_attributions': ['<a hr...",ChIJdWawtGhF5IkRCyrBlCXyyNQ,2.9,ChIJdWawtGhF5IkRCyrBlCXyyNQ,"[hospital, doctor, health, point_of_interest, ...",230.0,41.810925,-71.412546,41.812116,-71.411113,41.809416,-71.413813,True,"RH6P+9X Providence, Rhode Island",87HCRH6P+9X,41.810925,-71.412546
12133,OPERATIONAL,"248 State St #3a, Ellsworth, ME 04605, United ...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,Ellsworth Family Planning & Primary Care,"[{'height': 5312, 'html_attributions': ['<a hr...",ChIJe6l7-gnxrkwRcBfSY9uHdoQ,3.4,ChIJe6l7-gnxrkwRcBfSY9uHdoQ,"[doctor, point_of_interest, health, establishm...",5.0,44.551309,-68.429494,44.553176,-68.428355,44.550476,-68.431055,False,"HH2C+G6 Ellsworth,

In [62]:
gmaps_clinics = clinics[['lat','lon']].dropna().reset_index(drop=True)
gmaps_clinics['lat_lon'] = gmaps_clinics.apply(lambda x: [x['lat'],x['lon']],axis=1)

# df.to_feather("clinics_gmaps_605.feather")
gmaps_clinics.to_feather("gs://ohdo-post-roe-dev/data/clinics_gmaps.feather")

In [22]:

px.scatter_mapbox(
        clinics,
        lat="lat",
        lon="lon",
        size_max=15,
        height=600,
        zoom=3,
        title=f"Abortion Clincs | Google Places API | n={len(clinics)}"
    )

In [31]:
desinations = clinics[['lat','lon']].reset_index(drop=True)
desinations['type'] = "clinic"
desinations

,lat,lon,type
0,40.337842,-74.066505,clinic
1,40.689721,-73.982176,clinic
2,40.147447,-74.224413,clinic
3,40.552115,-74.287140,clinic
4,40.618421,-73.909974,clinic
...,...,...,...
600,41.810925,-71.412546,clinic
601,44.551309,-68.429494,clinic
602,44.722785,-67.439726,clinic
603,45.184282,-67.269722,clinic


In [46]:
origins = zip3[['lat','lon']].reset_index(drop=True)
origins['type'] = "origin"
df = pd.concat([origins, desinations]).reset_index(drop=True)
px.scatter_mapbox(
        df,
        lat="lat",
        lon="lon",
        size_max=15,
        height=600,
        zoom=3,
        color="type",
        color_discrete_sequence=["lightgrey", "magenta"]

    )

In [49]:
bq.clinics_query(case="main", n='515')

,zip5,state,lat,lon,lat_lon,population,adi_median
0,55901,MN,44.077657,-92.497063,"[44.077657, -92.4970628]",51917.0,48.0
1,55114,MN,44.966690,-93.195072,"[44.9666903, -93.1950724]",3100.0,37.0
2,03431,NH,42.969069,-72.301965,"[42.9690686, -72.301965]",25280.0,60.0
3,03104,NH,43.009552,-71.439782,"[43.0095519, -71.439782]",32798.0,39.0
4,07505,NJ,40.917008,-74.171936,"[40.9170078, -74.1719362]",2256.0,35.0
...,...,...,...,...,...,...,...
510,11372,NY,40.751719,-73.883670,"[40.7517193, -73.8836703]",66636.0,24.0
511,13057,NY,43.101955,-76.037845,"[43.1019554, -76.0378448]",14992.0,73.0
512,97210,OR,45.545484,-122.736853,"[45.5454839, -122.7368529]",10887.0,6.0
513,97239,OR,45.489938,-122.690453,"[45.4899383, -122.6904527]",14150.0,11.0


In [54]:
zip5 = bq.census_zip5_query()



,lat,lon
0,40.337842,-74.066505
1,40.689721,-73.982176
2,40.147447,-74.224413
3,40.552115,-74.287140
4,40.618421,-73.909974
...,...,...
11800,41.810925,-71.412546
12133,44.551309,-68.429494
12134,44.722785,-67.439726
12145,45.184282,-67.269722
